In [15]:
!pip install -q kor markdownify requests pydantic pydantic[email] openai colorama bs4 rich

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 3.0 MB/s eta 0:00:00


In [2]:
from datetime import date
from enum import Enum
from pydantic import BaseModel, ConfigDict


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
    )

    manufacturer: str
    series_name: str
    type_: AutomobileType
    is_electric: bool = False
    manufactured_date: date
    base_msrp_usd: float
    vin: str
    number_of_doors: int = 4
    registration_country: str | None = None
    license_plate: str | None = None


Modify your Automobile model to implement the following:

auto generate camel case aliases
- the field type_ in our model is provided as type in source data, and should also serialize to type.
- the data we receive contains the following field names that need to map to our own model field names - but we still want our camelized field names to be used for serialization. Account for that (without renaming the field names):
number_of_doors is provided as doors
- manufactured_date is provided as completionDate
- The field base_msrp_usd is provided as msrpUSD, and we want the serialization name to be baseMSRPUSD
- we want the JSON serialized output of manufactured_date to be this pattern: YYYY/MM/DD (e.g. 2020/01/01), but serializing to a Python dict should remain as a date object.

In [3]:
data_json = '''
{
    "manufacturer": "BMW",
    "seriesName": "M4",
    "type": "Convertible",
    "isElectric": false,
    "completionDate": "2023-01-01",
    "msrpUSD": 93300,
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "France",
    "licensePlate": "AAA-BBB"
}
'''



In [4]:
expected_serialized_dict = {
    'manufacturer': 'BMW',
    'series_name': 'M4',
    'type_': AutomobileType.convertible,
    'is_electric': False,
    'manufactured_date': date(2023, 1, 1),
    'base_msrp_usd': 93300.0,
    'vin': '1234567890',
    'number_of_doors': 2,
    'registration_country': 'France',
    'license_plate': 'AAA-BBB'
}

In [5]:
expected_serialized_dict_by_alias = {
    'manufacturer': 'BMW',
    'seriesName': 'M4',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': 'France',
    'licensePlate': 'AAA-BBB'
}

In [6]:
expected_serialized_json_by_alias = (
    '{"manufacturer":"BMW","seriesName":"M4","type":"Convertible",'
    '"isElectric":false,"manufacturedDate":"2023/01/01","baseMSRPUSD":93300.0,'
    '"vin":"1234567890","numberOfDoors":2,"registrationCountry":"France",'
    '"licensePlate":"AAA-BBB"}'
)

## Solution

In [7]:
from pydantic import Field, field_serializer
from pydantic.alias_generators import to_camel

class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    manufacturer: str
    series_name: str
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate")
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD",
        serialization_alias="baseMSRPUSD"
    )
    vin: str
    number_of_doors: int = Field(default=4, validation_alias="doors")
    registration_country: str | None = None
    license_plate: str | None = None

    @field_serializer("manufactured_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

In [8]:
car = Automobile.model_validate_json(data_json)
car

Automobile(manufacturer='BMW', series_name='M4', type_=<AutomobileType.convertible: 'Convertible'>, is_electric=False, manufactured_date=datetime.date(2023, 1, 1), base_msrp_usd=93300.0, vin='1234567890', number_of_doors=2, registration_country='France', license_plate='AAA-BBB')

In [9]:
assert car.model_dump() == expected_serialized_dict

In [10]:
assert car.model_dump(by_alias=True) == expected_serialized_dict_by_alias

In [11]:
assert car.model_dump_json(by_alias=True) == expected_serialized_json_by_alias

## ANOTHER EXAMPLE

In [13]:
from datetime import datetime, timedelta, timezone

from pydantic import BaseModel, ConfigDict, field_serializer
from pydantic_core.core_schema import FieldSerializationInfo, SerializerFunctionWrapHandler

class WithCustomEncoders(BaseModel):
    model_config = ConfigDict(ser_json_timedelta='iso8601')

    dt: datetime
    diff: timedelta
    diff2: timedelta

    @field_serializer('dt')
    def serialize_dt(self, dt: datetime, _info: FieldSerializationInfo):
        print(_info)
        return dt.timestamp()

    # The decorator below executes first
    @field_serializer('diff')
    def ssse(self, diff: timedelta, info: FieldSerializationInfo):
        print(info)
        return diff.total_seconds()

    @field_serializer('diff2', mode='wrap')
    @staticmethod
    def diff2_ser(diff2: timedelta, nxt: SerializerFunctionWrapHandler, info: FieldSerializationInfo):
        value = nxt(diff2)
        return value + 'postprocess'

m = WithCustomEncoders(
    dt=datetime(2032, 6, 1, tzinfo=timezone.utc), diff=timedelta(minutes=2),
    diff2=timedelta(minutes=1)
)

print(m.model_dump_json())

SerializationInfo(include=None, exclude=None, mode='json', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False)
SerializationInfo(include=None, exclude=None, mode='json', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False)
{"dt":1969660800.0,"diff":120.0,"diff2":"PT60Spostprocess"}


In [12]:
from typing import Dict, Any

from pydantic import BaseModel, model_serializer
from pydantic_core.core_schema import SerializerFunctionWrapHandler, SerializationInfo


class Model(BaseModel):
    x: str

    @model_serializer
    def ser_model(self, info: SerializationInfo):
        print(info)
        return {'x': f'xxxxxx {self.x}'}

    @model_serializer(mode='wrap')
    def ser_model_wrap(self, nxt: SerializerFunctionWrapHandler, info: SerializationInfo) -> Dict[str, Any]:
        print(info)
        return {'x': f'serialized {nxt(self)}'}


print(Model(x='test value').model_dump_json())

SerializationInfo(include=None, exclude=None, mode='json', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False)
{"x":"serialized {'x': 'test value'}"}


## Network Types
The networks module contains types for common network-related fields.

In [27]:
from pydantic import BaseModel, EmailStr, NameEmail, ValidationError,AnyUrl

class Model(BaseModel):
    email: EmailStr

try:
    Model(email="john.smith@some-domain")
except ValidationError as ex:
    print(ex)

1 validation error for Model
email
  value is not a valid email address: The part after the @-sign is not valid. It should have a period. [type=value_error, input_value='john.smith@some-domain', input_type=str]


In [19]:
class Model(BaseModel):
    email: NameEmail

In [20]:
m = Model(email="john.smith@some-domain.com")
m

Model(email=NameEmail(name='john.smith', email='john.smith@some-domain.com'))

In [21]:
m.email.name

'john.smith'

In [22]:
m.email.email

'john.smith@some-domain.com'

In [23]:
m.model_dump()

{'email': NameEmail(name='john.smith', email='john.smith@some-domain.com')}

In [24]:
m.model_dump_json()

'{"email":"john.smith <john.smith@some-domain.com>"}'

In [25]:
m = Model(email="John Smith <john.smith@some-domain.com>")
m

Model(email=NameEmail(name='John Smith', email='john.smith@some-domain.com'))

If the email-validator library isn't sufficient for your needs, you'll have to use custom validators and serializers.

Let's turn our attention to some of the URL types.

In [28]:
url = AnyUrl("https://www.google.com/search?q=pydantic")
url

Url('https://www.google.com/search?q=pydantic')

In [30]:
print(f"{url.scheme=}")
print(f"{url.host=}")
print(f"{url.path=}")
print(f"{url.query=}")
print(f"{url.port=}")
print(f"{url.username=}")
print(f"{url.password=}")

url.scheme='https'
url.host='www.google.com'
url.path='/search'
url.query='q=pydantic'
url.port=443
url.username=None
url.password=None


When you have a bare URL (no path), like this for example:
```
https://www.google.com

```
When Pydantic deserializes this URL, it ADDS a trailing /. This was not the case with Pydantic V1, so be careful if you define a URL field that you then later use to concatenate with a path - if you use string concatenation you could end up with a double slash at the start of your URL.

In [31]:
from pydantic import HttpUrl

class ExternalAPI(BaseModel):
    root_url: HttpUrl

api = ExternalAPI(root_url="https://api.myserver.com")
api

ExternalAPI(root_url=Url('https://api.myserver.com/'))

**Resource:

-- http://windcf.com/archives/102

-- https://zenn.dev/yosemat/articles/6834cfc8de0d86